In [3]:
import pandas as pd
import sys
sys.path.append(r'C:\Users\Feder-PC\Desktop\ds_challenge_olist\src')

from data_loader import DataLoader
from model import HybridRecommender
from evaluate import RecommendationEvaluator

# Load data
loader = DataLoader()
data = loader.load_data()
data['purchase_date'] = pd.to_datetime(data['purchase_date'])

# Sort by date
data = data.sort_values('purchase_date')

# Split dataset: 70% train, 15% validation, 15% test
total_records = len(data)
train_end_idx = int(total_records * 0.70)
validation_end_idx = int(total_records * 0.85)

train_set = data.iloc[:train_end_idx].copy()
validation_set = data.iloc[train_end_idx:validation_end_idx].copy()
test_set = data.iloc[validation_end_idx:].copy()

print(f"Train set: {len(train_set):,} records ({len(train_set)/total_records:.1%})")
print(f"Validation set: {len(validation_set):,} records ({len(validation_set)/total_records:.1%})")
print(f"Test set: {len(test_set):,} records ({len(test_set)/total_records:.1%})")
print(f"Date ranges:")
print(f"  Train: {train_set['purchase_date'].min()} to {train_set['purchase_date'].max()}")
print(f"  Validation: {validation_set['purchase_date'].min()} to {validation_set['purchase_date'].max()}")
print(f"  Test: {test_set['purchase_date'].min()} to {test_set['purchase_date'].max()}")

# Define window_days to test
window_days_list = [i for i in range(60, 240, 20)]

# Initialize evaluator
evaluator = RecommendationEvaluator()

# Store results
results_list = []

# Evaluate each window_days value
for window_days in window_days_list:
    print(f"\nEvaluating window_days = {window_days}")
    
    results = evaluator.evaluate_simple(
        train_data=train_set,
        test_data=validation_set,
        model_class=HybridRecommender,
        k=5,
        window_days=window_days
    )
    
    # Add window_days to results
    results['window_days'] = window_days
    results_list.append(results)

# Convert to DataFrame
results_df = pd.DataFrame(results_list)

# Reorder columns
columns_order = [
    'window_days',
    'new_users', 
    'new_users_orders',
    'returning_customers',
    'returning_customers_orders', 
    'hits_new_users_orders',
    'hits_returning_customers_orders',
    'calculate_precision_at_k_for_new_users',
    'calculate_precision_at_k_for_returning'
]

results_df = results_df[columns_order]

print("\n" + "="*80)
print("FINAL RESULTS DATAFRAME")
print("="*80)
results_df

Train set: 78,855 records (70.0%)
Validation set: 16,897 records (15.0%)
Test set: 16,898 records (15.0%)
Date ranges:
  Train: 2016-09-04 00:00:00 to 2018-04-14 00:00:00
  Validation: 2018-04-14 00:00:00 to 2018-06-19 00:00:00
  Test: 2018-06-19 00:00:00 to 2018-09-03 00:00:00

Evaluating window_days = 240
📊 EVALUATION RESULTS (k=5, window_days=240)
New users: 14,119
New users orders: 16,445
Returning customers: 347
Returning customers orders: 452
Hits new users orders: 242
Hits returning customers orders: 8
Precision@5 for new users: 0.0034
Precision@5 for returning: 0.0046


FINAL RESULTS DATAFRAME


,window_days,new_users,new_users_orders,returning_customers,returning_customers_orders,hits_new_users_orders,hits_returning_customers_orders,calculate_precision_at_k_for_new_users,calculate_precision_at_k_for_returning
0,240,14119,16445,347,452,242,8,0.003428,0.004611


In [4]:
# Create combined training set (train + validation)
train_validation_set = pd.concat([train_set, validation_set]).sort_values('purchase_date')

print(f"Combined train+validation: {len(train_validation_set):,} records")
print(f"Test set: {len(test_set):,} records")
print(f"Test period: {test_set['purchase_date'].min()} to {test_set['purchase_date'].max()}")

# Get unique test dates
test_dates = sorted(test_set['purchase_date'].dt.date.unique())
print(f"Total test days: {len(test_dates)}")

# Initialize
evaluator = RecommendationEvaluator()
window_days = 140
daily_results = []

# Rolling evaluation day by day
for i, test_date in enumerate(test_dates):
    test_date_dt = pd.to_datetime(test_date)
    
    print(f"\nDay {i+1}/{len(test_dates)}: {test_date}")
    
    # Get test data for this specific day
    day_test_data = test_set[test_set['purchase_date'].dt.date == test_date]
    
    if len(day_test_data) == 0:
        print(f"  No orders on {test_date}, skipping...")
        continue
    
    # Build expanding training set
    if i == 0:
        # First day: only train+validation
        expanding_train = train_validation_set.copy()
    else:
        # Add all previous test days to training
        previous_test_data = test_set[test_set['purchase_date'].dt.date < test_date]
        expanding_train = pd.concat([train_validation_set, previous_test_data]).sort_values('purchase_date')
    
    print(f"  Training records: {len(expanding_train):,}")
    print(f"  Test records: {len(day_test_data):,}")
    
    # Evaluate this day
    try:
        results = evaluator.evaluate_simple(
            train_data=expanding_train,
            test_data=day_test_data,
            model_class=HybridRecommender,
            k=5,
            window_days=window_days
        )
        
        # Add date and window_days
        results['test_date'] = test_date
        results['window_days'] = window_days
        daily_results.append(results)
        
    except Exception as e:
        print(f"  Error evaluating {test_date}: {e}")
        continue

# Convert to DataFrame
daily_results_df = pd.DataFrame(daily_results)

# Reorder columns
columns_order = [
    'test_date',
    'window_days',
    'new_users', 
    'new_users_orders',
    'returning_customers',
    'returning_customers_orders', 
    'hits_new_users_orders',
    'hits_returning_customers_orders',
    'calculate_precision_at_k_for_new_users',
    'calculate_precision_at_k_for_returning'
]

if len(daily_results_df) > 0:
    daily_results_df = daily_results_df[columns_order]

print("\n" + "="*80)
print("DAILY ROLLING EVALUATION RESULTS")
print("="*80)
print(f"Total days evaluated: {len(daily_results_df)}")
daily_results_df

Combined train+validation: 95,752 records
Test set: 16,898 records
Test period: 2018-06-19 00:00:00 to 2018-09-03 00:00:00
Total test days: 73

Day 1/73: 2018-06-19
  Training records: 95,752
  Test records: 10
📊 EVALUATION RESULTS (k=5, window_days=140)
New users: 10
New users orders: 10
Returning customers: 0
Returning customers orders: 0
Hits new users orders: 0
Hits returning customers orders: 0
Precision@5 for new users: 0.0000
Precision@5 for returning: 0.0000


Day 2/73: 2018-06-20
  Training records: 95,762
  Test records: 251
📊 EVALUATION RESULTS (k=5, window_days=140)
New users: 207
New users orders: 238
Returning customers: 10
Returning customers orders: 13
Hits new users orders: 1
Hits returning customers orders: 0
Precision@5 for new users: 0.0010
Precision@5 for returning: 0.0000


Day 3/73: 2018-06-21
  Training records: 96,013
  Test records: 264
📊 EVALUATION RESULTS (k=5, window_days=140)
New users: 226
New users orders: 257
Returning customers: 7
Returning customers o

,test_date,window_days,new_users,new_users_orders,returning_customers,returning_customers_orders,hits_new_users_orders,hits_returning_customers_orders,calculate_precision_at_k_for_new_users,calculate_precision_at_k_for_returning
0,2018-06-19,140,10,10,0,0,0,0,0.000000,0.0
1,2018-06-20,140,207,238,10,13,1,0,0.000966,0.0
2,2018-06-21,140,226,257,7,7,2,0,0.001770,0.0
3,2018-06-22,140,173,200,6,6,0,0,0.000000,0.0
4,2018-06-23,140,143,163,7,9,0,0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
68,2018-08-26,140,71,79,2,3,1,0,0.002817,0.0
69,2018-08-27,140,63,68,3,3,0,0,0.000000,0.0
70,2018-08-28,140,36,39,3,6,1,0,0.005556,0.0
71,2018-08-29,140,11,16,0,0,0,0,0.000000,0.0
